In [19]:
%%capture
!pip install librosa
!apt-get -y install libsndfile-dev ffmpeg
!pip install lmdb, tqdm

In [1]:
import librosa

def get_mfcc(path, start=0.0, duration = None):
    x, sr = librosa.load(path)

    return librosa.feature.mfcc(x, sr=sr)

## GTZAN

In [14]:
GTZAN_path = "../subtasks/GTZAN/"

from source.sound_transforms import log_mel_spectrogram
import librosa
import os
from tqdm.notebook import tqdm
import lmdb

db = []


env = lmdb.open('data/GTZAN/filename_to_log_mel_spectrograms.lmdb', subdir=False, map_size=1e12 )

for (dirpath, dirnames, filenames) in os.walk(GTZAN_path):
    for filename in tqdm(filenames):
        path = os.path.join(dirpath, filename)
        
        ext = os.path.splitext(path)[-1]
        
        genre = dirpath.split("/")[-1]
        
        if ext.lower() == ".wav":
            track_id = filename.encode()
            
            x, sr = librosa.load(path, sr=None)
            features = log_mel_spectrogram(x,sr)
            
            with env.begin(write=True) as txn:
                txn.put(track_id, features.tobytes())
            
            db.append((filename, genre))
            
len(db)

1000

In [15]:
import numpy as np

with open("data/GTZAN/filename_to_genre.csv", "w") as f:
    f.write("filename, genre \n")
    for row in db:
        f.write(row[0] + ", " + row[1] + "\n")

In [16]:
env.stat()

{'psize': 4096,
 'depth': 2,
 'branch_pages': 1,
 'leaf_pages': 16,
 'overflow_pages': 375201,
 'entries': 1000}

## EmoMusic 

In [17]:
import csv

db_info = []
with open("../subtasks/emoMusic/annotations/songs_info.csv", "r" , encoding='utf-8') as f:
    
    for row in csv.reader(f):
        db_info.append(row)   

dict_static = {}
with open("../subtasks/emoMusic/annotations/static_annotations.csv", "r" , encoding='utf-8') as f:
    
    for row in csv.reader(f):
        dict_static[row[0]] = row[1:]

In [22]:
dict_static

{'song_id': ['mean_arousal', 'std_arousal', 'mean_valence', 'std_valence'],
 '2': ['3.1', '0.99443', '3', '0.66667'],
 '3': ['3.5', '1.8409', '3.3', '1.7029'],
 '4': ['5.7', '1.4944', '5.5', '1.7159'],
 '5': ['4.4', '2.1187', '5.3', '1.9465'],
 '7': ['5.8', '1.5492', '6.4', '1.7764'],
 '8': ['3.2', '1.4757', '4.8', '1.6193'],
 '10': ['4', '1.7638', '4.7', '1.9465'],
 '12': ['5.5', '2.0138', '5.8', '1.9889'],
 '13': ['3.2', '1.4757', '4', '1.7638'],
 '17': ['4.4', '1.8974', '6', '2.1602'],
 '18': ['4.8', '1.8135', '3.9', '1.4491'],
 '19': ['5.9', '1.7288', '4.3', '2.0028'],
 '20': ['5.4', '2.6331', '6.5', '2.2236'],
 '21': ['6.6', '0.96609', '6.3', '1.4944'],
 '22': ['4', '1.4142', '4.8', '1.5492'],
 '24': ['5.3', '1.7029', '3.9', '1.8529'],
 '25': ['7.9', '0.73786', '4.7', '2.6687'],
 '31': ['4.7', '1.6364', '3.8', '1.6193'],
 '32': ['4.2', '1.8738', '3.3', '1.567'],
 '35': ['4.8', '2.2509', '6.5', '2.2236'],
 '37': ['6.6', '2.2211', '5.3', '2.1108'],
 '39': ['7.3', '1.9465', '5.4', '2

In [18]:
train = []
test = []

for row in db_info:
    
    obj = {
        'song_id' : row[0],
        'filename': row[1],
        'emo_values': dict_static[row[0]]
    }
    
    if row[-1] == 'development':
        train.append(obj)
    elif row[-1] == 'evaluation':
        test.append(obj)
    
len(train), len(test)

(619, 125)

In [33]:
from tqdm.notebook import tqdm

emoMusic_path = "../subtasks/emoMusic/clips_45seconds/"
env = lmdb.open('data/emoMusic/filename_to_log_mel_spectrograms.lmdb', subdir=False, map_size=1e12 )

for row in tqdm(db_info[1:]):
    song_id = row[0]
    filename = row[1].strip()
    path = emoMusic_path + filename.strip()

    x, sr = librosa.load(path, sr=None)
    features = log_mel_spectrogram(x,sr)
    
    track_id = song_id.encode()
    
    with env.begin(write=True) as txn:
        txn.put(track_id, features.tobytes())
    

In [32]:
env.stat()

{'psize': 4096,
 'depth': 2,
 'branch_pages': 1,
 'leaf_pages': 23,
 'overflow_pages': 794066,
 'entries': 1744}

In [21]:
import warnings
import tqdm.notebook as tqdm
warnings.simplefilter("ignore")

emoMusic_path = "../subtasks/emoMusic/clips_45seconds/"
env = lmdb.open('data/emoMusic/filename_to_log_mel_spectrograms.lmdb', subdir=False, map_size=1e12 )

for item in tqdm.tqdm(train+test):
    path = emoMusic_path + item['filename'].strip()
    
    x, sr = librosa.load(path, sr=None)
    features = log_mel_spectrogram(x,sr)
    
    track_id = item['filename'].encode()
    
    with env.begin(write=True) as txn:
        txn.put(track_id, features.tobytes())
    
    #item['emo_values'] = np.array(item['emo_values']).astype(np.float)
    
    item['emo_values']

KeyboardInterrupt: 

In [20]:
import warnings
import numpy as np
import tqdm.notebook as tqdm
warnings.simplefilter("ignore")

emoMusic_path = "../subtasks/emoMusic/clips_45seconds/"

for item in tqdm.tqdm(train+test):
    path = emoMusic_path + item['filename'].strip()
    item['path'] = path
    
# Small corrections in the objects
for item in train+test:
    item['emo_values'] = np.array(item['emo_values']).astype(np.float)

In [6]:
import numpy as np

with open("data/emoMusic/train_path.npy", "wb") as f:
    np.save(f, train)
    
with open("data/emoMusic/test_path.npy", "wb") as f:
    np.save(f, test)

## Speech/Music Discrimination

In [4]:
MS_path = "../subtasks/music_speech/music_speech/"
import os
import numpy as np
from tqdm.notebook import tqdm
import lmdb
import librosa
from source.sound_transforms import log_mel_spectrogram


env = lmdb.open('data/music_speech/filename_to_log_mel_spectrograms.lmdb', subdir=False, map_size=1e12 )

dataset = []
for (dirpath, dirnames, filenames) in os.walk(MS_path):
    
    for filename in tqdm(filenames):
        if ".wav" in filename:
            binary_class = dirpath.split("/")[-1]
            path = os.path.join(dirpath, filename)
            
            x, sr = librosa.load(path, sr=None)
            features = log_mel_spectrogram(x,sr)

            track_id = filename.encode()

            with env.begin(write=True) as txn:
                txn.put(track_id, features.tobytes())
            
            item = {
                "id" : filename,
                "class" : binary_class,
            }
            dataset.append(item)

len(dataset)

128

In [5]:
import numpy as np

with open("data/music_speech/filename_to_class.csv", "w") as f:
    f.write("filename, class \n")
    for row in dataset:
        f.write(row['id'] + ", " + row['class'] + "\n")

In [11]:
import numpy as np

with open("data/music_speech/path.npy", "wb") as f:
    np.save(f, dataset)

## Deezer Mood Detection

In [12]:
import csv

db_info = []
with open("../subtasks/deezer_mood/deezer_mood_detection_dataset/train.csv", "r" , encoding='utf-8') as f:
    
    for row in csv.reader(f):
        if "valence" in row:
            continue
        db_info.append(row[2:5]) 
        
with open("../subtasks/deezer_mood/deezer_mood_detection_dataset/validation.csv", "r" , encoding='utf-8') as f:
    
    for row in csv.reader(f):
        if "valence" in row:
            continue
        db_info.append(row[2:5]) 
        
with open("../subtasks/deezer_mood/deezer_mood_detection_dataset/test.csv", "r" , encoding='utf-8') as f:
    
    for row in csv.reader(f):
        if "valence" in row:
            continue
        db_info.append(row[2:5]) 
        
        
#x = np.array(db_info[1:]).astype(np.float)

db_info[0]

['TRJIEJX12903CF2202', '-0.682725080497', '0.316757791845']

In [11]:
failed = set()

with open("data/deezer/failed.csv", "r") as f:
    
    for row in f.readlines():
        msd_id = row.strip()
        failed.add(msd_id)
        
len(failed)

311

In [17]:
deezer = set()

with open("data/deezer/msd_id_to_emotion.csv", "r") as f:
    
    for row in f.readlines():
        msd_id = row[0].strip()
        deezer.add(msd_id)
        
deezer.intersection(failed)

set()

In [14]:
count = 0
for row in db_info:
    msd_id = row[0]
    
    if msd_id in failed:
        count += 1
        
count

311

In [16]:
import numpy as np

with open("data/deezer/msd_id_to_emotion.csv", "w") as f:
    f.write("msd_id, valence, arousal \n")
        
    for row in db_info:
        
        if row[0] in failed:
            continue
        else:
            f.write(row[0] + ", " + row[1] +", " + row[2] + "\n")

In [3]:
import lmdb
import csv

from tqdm.notebook import tqdm

db=[]
missing_count = 0

with open("data/deezer/msd_id_to_emotion.csv", "r") as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader, None)

    for row in tqdm(reader):

        msd_id = row[0].encode()
        
        song_env = lmdb.open("data/MSD/MSD_ID_to_log_mel_spectrogram.lmdb", 
                                  subdir=False, readonly=True, lock=False,
                                  readahead=False, meminit=False, max_readers=1)
        
        with song_env.begin(write=False) as txn:
            data = txn.get(msd_id[:17])
            
        if data is None:
            missing_count += 1
            print(msd_id)
        
missing_count

b'TRAGFVT128F92ECB5D'
b'TRTPCNQ128F42A3B02'
b'TRRTNCU128F92F1397'
b'TRLBUWT128F92EFAE0'
b'TRGVGAU128F425AD55'
b'TRRCVKI128F92F0410'
b'TRREKCT12903D056A8'
b'TRMRIKU128F92FA7C3'
b'TRRBYQJ128F92FD3F2'
b'TRHJGCA128F92FD364'
b'TRGTZBU128F42BC23D'
b'TRWMZQL128F9306DCB'
b'TRHMZNR128F92F078E'
b'TRJNOAC128F9327377'
b'TRGDLXH128F9301435'
b'TRZXWKQ128F92F121E'
b'TROWFCD128F42626BF'
b'TRZALAV128F1484413'
b'TRGIZCC128F1484415'
b'TRWPZIT12903D096D3'
b'TRRRWWF128F4279C7C'
b'TRBDMDS128F930B8F1'
b'TRGGWBK128F42B7711'
b'TRZAMPP128F426C205'
b'TRBUPCV128F92F694E'
b'TRALJVQ128F428C764'
b'TRBOXER128F4281C8B'
b'TRUQEKG128F9310309'
b'TRUUQND128F93104A3'
b'TRGPSXC128F93104A4'
b'TRMZEXY128F93104A6'
b'TRNXTVM128F4281CA0'
b'TRGRAJE128F4284B6A'
b'TRHVZFK128F42AC2FB'
b'TRIEPIJ128F9311AD8'
b'TRMTZDR128F93169BF'
b'TRCBSXB128F933ACAD'
b'TRCLWUA128F933D438'
b'TRELTDZ128F932149A'
b'TRQJZAZ128F4274857'
b'TRMSHQN128E0792879'
b'TROLCEJ128F4284281'
b'TRMALIE128F4284682'
b'TRHSIRO128C71962D1'
b'TRNVCFD128EF34DD82'
b'TRZSCNY1

311

# UrbanSound8K

In [20]:
csv_path = '/nfs/subtasks/urbansound8k/UrbanSound8K/metadata/UrbanSound8K.csv'
urbansound=[]

with open("../subtasks/MagnaTagATune/clip_info_final.csv", "r" , encoding='utf-8') as f:
    for row in csv.reader(f, delimiter='\t'):
        urbansound.append(row)
        
urbansound[0]

['clip_id',
 'track_number',
 'title',
 'artist',
 'album',
 'url',
 'segmentStart',
 'segmentEnd',
 'original_url',
 'mp3_path']

In [29]:
import os

audio_path = '/nfs/subtasks/urbansound8k/UrbanSound8K/audio/'

splits = set()

for (dirpath, dirnames, filenames) in os.walk(audio_path):
    for filename in filenames:
        if "wav" in filename:
            fold = dirpath.split("/")[-1]
            splits.add(fold)
            
splits

{'fold1',
 'fold10',
 'fold2',
 'fold3',
 'fold4',
 'fold5',
 'fold6',
 'fold7',
 'fold8',
 'fold9'}

## MagnaTagATune

In [1]:
import csv

db_info = []
with open("../subtasks/MagnaTagATune/clip_info_final.csv", "r" , encoding='utf-8') as f:
    
    for row in csv.reader(f, delimiter='\t'):
        db_info.append(row) 
        
db_anno = []
with open("../subtasks/MagnaTagATune/annotations_final.csv", "r" , encoding='utf-8') as f:
    
    for row in csv.reader(f, delimiter='\t'):
        db_anno.append(row) 
        
len(db_anno), len(db_info)

(25864, 31383)

In [2]:
import lmdb
import json
import sys
import warnings
import tqdm.notebook as tqdm
import numpy as np
warnings.simplefilter("ignore")

base_path = "../subtasks/MagnaTagATune/mp3/"

def create_item(row):
    path = base_path + row[-1]
    tags = row[1:-1]
    
    try:
        mfcc = get_mfcc(path)
        item = {
            "mfcc": mfcc.tolist(),
            "tags": tags, 
        }
    except KeyboardInterrupt as e:
        raise e
    except Exception as e:
        print("Item %s discarded"%row[0])
        return None
    
    
    return json.dumps(item).encode()

env = lmdb.open('data/MTAT/', map_size=2*len(db_anno)*sys.getsizeof(create_item(db_anno[1])))

start_index = 25742

for i, row in enumerate(tqdm.tqdm(db_anno[start_index:])):
    with env.begin(write=True) as txn:
        idx = start_index + i
        str_id = '{:08}'.format(idx).encode("ascii")
        
        
        item = create_item(row)
        if item is not None:
            txn.put(str_id, item)

Item 2 discarded


Item 58459 discarded
Item 58460 discarded
Item 58467 discarded
Item 58470 discarded
Item 58471 discarded
Item 58472 discarded
Item 58473 discarded
Item 58474 discarded
Item 58476 discarded
Item 58477 discarded
Item 58479 discarded
Item 58481 discarded
Item 58484 discarded
Item 58486 discarded
Item 58488 discarded
Item 58490 discarded
Item 58491 discarded
Item 58502 discarded
Item 58503 discarded
Item 58504 discarded
Item 58506 discarded
Item 58508 discarded
Item 58509 discarded
Item 58510 discarded
Item 58511 discarded
Item 58513 discarded
Item 58514 discarded
Item 58515 discarded
Item 58517 discarded
Item 58525 discarded
Item 58526 discarded
Item 58527 discarded
Item 58528 discarded
Item 58530 discarded
Item 58531 discarded
Item 58538 discarded
Item 58539 discarded
Item 58553 discarded
Item 58554 discarded
Item 58555 discarded
Item 58559 discarded
Item 58560 discarded
Item 58561 discarded
Item 58565 discarded
Item 58566 discarded
Item 58569 discarded
Item 58572 discarded
Item 58573 di

In [3]:
with env.begin() as txn:
    length = txn.stat()['entries']
    
length, len(db_anno)

(25861, 25864)

In [20]:
import librosa
import stream
from multiprocessing import Lock

from joblib import Parallel, delayed
from source.io.pb_item import pb_Item

def get_mfcc(path, start=0.0, duration = None):
    x, sr = librosa.load(path, sr=None, offset=start, duration=duration)

    return librosa.feature.mfcc(x, sr=sr)

def create_pb_item(row):
    path = base_path + row[-1]
    tags = row[1:-1]
    tags = [int(x) for x in tags]
    
    try:
        mfcc = get_mfcc(path)
        pb_item = pb_Item(features=mfcc,
                          seq_targets=tags)
        
        with stream.open("data/MTAT/data.bin", "ab") as ostream:
            ostream.write(pb_item.get_pb_obj())

        
    except KeyboardInterrupt as e:
        raise e
    except Exception as e:
        print(e)
        print("Item %s discarded"%row[0])

start_index = 1

with stream.open("data/MTAT/data.bin", "wb") as ostream:
    pass

Parallel(n_jobs=-1, backend='multiprocessing')\
         (delayed(create_pb_item)(row=row)\
          for row in tqdm.tqdm(db_anno[start_index:]))

print("Done")


Item 35644 discarded

Item 55753 discarded

Item 57881 discarded

Done


In [7]:
from source.utils import Timer

i=0

with Timer() as t:
    with stream.open("data/MTAT/test.bin", "rb") as istream:
        for data in istream:
            x = pb_Item()
            x.load_from_string(data)
            i += 1

print(t)
i

0.211700439453125


64

In [23]:
! ls -lh data/MTAT/

total 11G
-rw-r--r--. 1 nobody nogroup  81M Feb  7 17:54 data_1000.npy
-rw-r--r--. 1 nobody nogroup 1.9G Feb 17 18:12 data.bin
-rw-r--r--. 1 nobody nogroup 8.9G Feb 17 17:09 data.mdb
-rw-r--r--. 1 nobody nogroup    0 Feb  8 14:53 data.npy
-rw-r--r--. 1 nobody nogroup 8.0K Feb 17 17:31 lock.mdb


In [4]:
from source.sound_utils import get_mfcc
import stream
from source.io.pb_item import pb_Item

base_path = "../subtasks/MagnaTagATune/mp3/"

def write_item(item):
    
    path = base_path + row[-1]
    tags = [int(x) for x in row[1:-1]]
    
    try:
        mfcc = get_mfcc(path)
        pb_item = pb_Item(features=mfcc, seq_targets=tags)
        with stream.open("data/MTAT/test.bin", "ab") as ostream:
            ostream.write(pb_item.get_pb_obj())
            
    except KeyboardInterrupt as e:
        raise e
    except Exception as e:
        print("Item %s discarded"%row[0])

In [5]:
from source.io.parallel_processing import parallel_extract

parallel_extract(db_anno[25800:], write_item)


Done


## GiantSteps

In [51]:
import pandas as pd
file_errors_location = '../git/giantsteps-key-dataset/sources.xlsx'
df = pd.read_excel(file_errors_location)
df.shape

(604, 8)

In [94]:
import json

path_anno  = "../git/giantsteps-key-dataset/annotations/jams/"
path_audio = "../git/giantsteps-key-dataset/audio/"

dataset = []
for index, row in df.iterrows():
    track_id = row['TRACK']
    
    path_annotation = path_anno + str(track_id) + ".LOFI.jams"
    audio_path = path_audio + str(track_id) + ".LOFI.wav"
    
    with open(path_annotation, "r") as f:
        annotation = json.load(f)
        audio_class = annotation['annotations'][0]['data'][0]['value']
    
    item = {
                "class" : audio_class,
                "path" : audio_path,
            }
    dataset.append(item)
    
len(dataset)

604

In [95]:
import numpy as np

with open("data/GiantSteps/key_path.npy", "wb") as f:
    np.save(f, dataset)

In [107]:
path_audio = "../git/giantsteps-tempo-dataset/audio/"
path_annotations = "../git/giantsteps-tempo-dataset/annotations/jams/"

dataset = []
with open("../git/giantsteps-tempo-dataset/splits/files.txt", "r") as f:
    for x in f.readlines():
        x = x.strip()
        audio_path = path_audio + x.replace("mp3", "wav")
        anno_path  = path_annotations + x.replace("mp3", "jams")
        
        with open(anno_path, "r") as f:
            annotation = json.load(f)
            target = annotation['annotations'][0]['data'][0]['value']
            
        item = {
                    "target" : np.array([target]),
                    "path" : audio_path,
                }
        dataset.append(item)
len(dataset)

664

In [108]:
# Normalize target

list_targets = [item['target'] for item in dataset]

avg, std = np.mean(list_targets), np.std(list_targets)

for item in dataset:
    item['target'] = (item['target'] - avg)/std

In [109]:

np.mean(list_targets), np.std(list_targets)

(136.6594954819277, 28.33103134639793)

In [106]:
import numpy as np

with open("data/GiantSteps/tempo_path.npy", "wb") as f:
    np.save(f, dataset)

In [128]:
import librosa

x, sr = librosa.load(dataset[0]['path'], sr = None)

In [129]:
np.max(x), np.min(x)

(0.9885864, -0.9885864)

In [130]:
from pydub import AudioSegment

song = AudioSegment.from_wav(dataset[0]['path'])
song.set_frame_rate(sr)

In [131]:
y = np.array(song.get_array_of_samples())

In [133]:
y_norm = y / 32768.0

np.linalg.norm(x-y_norm)

0.0

In [4]:
import torch

(torch.rand(5,4) > 0.5).int()

tensor([[0, 0, 1, 0],
        [1, 0, 0, 1],
        [0, 0, 1, 1],
        [0, 0, 0, 1],
        [1, 0, 0, 1]], dtype=torch.int32)